# Bootstrap

## Load the JSON data

In [57]:
all_sets_filename = os.path.join('data', 'AllSets.json')

In [61]:
# Uncomment the following lines to update the date file
#import urllib
#urllib.urlretrieve ('http://hearthstonejson.com/json/AllSets.json', all_sets_filename)

In [62]:
import json
import os.path

with open(all_sets_filename) as fp:
        all_card_sets = json.load(fp, encoding='utf-8')

## Collectible cards

In [63]:
all_cards = sum((v for k, v in all_card_sets.items()
                 if k not in ('Debug', 'Credits', 'Missions', 'System')), list())
# Select only collectible cards
all_collectible_cards = [card for card in all_cards if u'collectible' in card and card['collectible']]
# Remove heroes
all_collectible_cards = [card for card in all_collectible_cards
                         if 'type' in card and card['type'] != 'Hero']
len(all_collectible_cards)

535

## Card tags

In [64]:
tags = set()
for card in all_collectible_cards:
    tags.update(set(card.keys()))
tags

{u'artist',
 u'attack',
 u'collectible',
 u'cost',
 u'durability',
 u'elite',
 u'faction',
 u'flavor',
 u'health',
 u'howToGet',
 u'howToGetGold',
 u'id',
 u'inPlayText',
 u'mechanics',
 u'name',
 u'playerClass',
 u'race',
 u'rarity',
 u'text',
 u'type'}

In [71]:
# Only interested in these tags for pricer purpouses
interest_tags = {u'attack', u'cost', u'durability', u'health', u'id', u'mechanics', u'name',
                 u'playerClass', u'text', u'type'}
all_collectible_cards = [{k: v for k, v in card.items() if k in interest_tags}
                         for card in all_collectible_cards]

## Card mechanics

In [72]:
mechanics = set()
for card in all_collectible_cards:
    if 'mechanics' in card:
        mechanics.update(set(card['mechanics']))
mechanics

{u'AdjacentBuff',
 u'AffectedBySpellPower',
 u'Aura',
 u'Battlecry',
 u'Charge',
 u'Combo',
 u'Deathrattle',
 u'Divine Shield',
 u'Enrage',
 u'Freeze',
 u'HealTarget',
 u'ImmuneToSpellpower',
 u'Poisonous',
 u'Secret',
 u'Silence',
 u'Spellpower',
 u'Stealth',
 u'Taunt',
 u'Windfury'}

## Card types

In [73]:
types = set()
for card in all_collectible_cards:
    if 'type' in card:
        types.add(card['type'])
types

{u'Minion', u'Spell', u'Weapon'}

## Big data cards

In [307]:
import pandas
all_collectible_cards_df = pandas.DataFrame(all_collectible_cards)

# Set/modify the pricing attributes
all_collectible_cards_df['intrinsic'] = -1

all_collectible_cards_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 535 entries, 0 to 534
Data columns (total 11 columns):
attack         357 non-null float64
cost           535 non-null int64
durability     18 non-null float64
health         339 non-null float64
id             535 non-null object
mechanics      279 non-null object
name           535 non-null object
playerClass    306 non-null object
text           517 non-null object
type           535 non-null object
intrinsic      535 non-null int64
dtypes: float64(3), int64(2), object(6)
memory usage: 37.6+ KB


# Some statistics

# Vanilla test

In [308]:
vanilla_minions_df = pandas.DataFrame(
    all_collectible_cards_df[(all_collectible_cards_df['type'] == 'Minion') &
                             (all_collectible_cards_df['text'].isnull())])
vanilla_minions_df.info()
print(type(vanilla_minions_df))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 238 to 517
Data columns (total 11 columns):
attack         14 non-null float64
cost           14 non-null int64
durability     0 non-null float64
health         14 non-null float64
id             14 non-null object
mechanics      0 non-null object
name           14 non-null object
playerClass    0 non-null object
text           0 non-null object
type           14 non-null object
intrinsic      14 non-null int64
dtypes: float64(3), int64(2), object(6)
memory usage: 1008.0+ bytes
<class 'pandas.core.frame.DataFrame'>


In [309]:
vanilla_minions_df['mechanics'].unique()

array([nan], dtype=object)

In [310]:
vanilla_minions_df[['name', 'cost', 'attack', 'health']]

,name,cost,attack,health
238,Wisp,0,1,1
289,Bloodfen Raptor,2,3,2
293,Boulderfist Ogre,6,6,7
295,Chillwind Yeti,4,4,5
299,Core Hound,7,9,5
347,Magma Rager,3,5,1
356,Murloc Raider,1,2,1
361,Oasis Snapjaw,4,2,7
368,River Crocolisk,2,2,3
400,War Golem,7,7,7


In [312]:
import numpy

a = vanilla_minions_df.as_matrix(['attack', 'health', 'intrinsic'])
b = vanilla_minions_df.as_matrix(['cost'])
cost_per_point = numpy.linalg.lstsq(a, b)[0]
cost_per_point

array([[ 0.58355876],
       [ 0.49202403],
       [ 0.75051715]])

In [313]:
card_value = cost_per_point[2][0]
vanilla_minions_df['value'] = numpy.dot(a, cost_per_point).T[0]
vanilla_minions_df['boost'] = (vanilla_minions_df['value'] + card_value) / \
    (vanilla_minions_df['cost'] + card_value)
vanilla_minions_df[['name', 'cost', 'value', 'boost']]

,name,cost,value,boost
238,Wisp,0,0.325066,1.433122
289,Bloodfen Raptor,2,1.984207,0.994258
293,Boulderfist Ogre,6,6.195004,1.028887
295,Chillwind Yeti,4,4.043838,1.009228
299,Core Hound,7,6.961632,0.995050
347,Magma Rager,3,2.659301,0.909159
356,Murloc Raider,1,0.908624,0.947801
361,Oasis Snapjaw,4,3.860769,0.970691
368,River Crocolisk,2,1.892672,0.960979
400,War Golem,7,6.778562,0.971429


## 2-cost minions

In [118]:
two_cost_minions_df = all_collectible_cards_df[(all_collectible_cards_df['type'] == 'Minion') &
                                               (all_collectible_cards_df['cost'] == 2)]
print('2-cost minion mean attack: {:.2f}'.format(two_cost_minions_df.attack.mean()))
print('2-cost minion mean health: {:.2f}'.format(two_cost_minions_df.health.mean()))

2-cost minion mean attack: 1.88
2-cost minion mean health: 2.45
